# Анализ специальностей

In [2]:
import numpy as np
import pandas as pd
import re
from ast import literal_eval
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter

In [3]:
df = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/merge/doctors.csv')

In [ ]:
df

In [ ]:
df.info()

In [7]:
def to_list(cell):
    if pd.isna(cell): # если строка пустая то функция возвращает Nan
        return []
    s = str(cell).strip() # приводим значение к строке и убираем пробелы по краям, дальше всё обрабатываем как текст
    if s.startswith('[') and s.endswith(']'): # смотрим выглядит ли строка как список
        try:
            lst = literal_eval(s) # безопасно превращаем строку, записанную синтаксисом Python, в объект Python. например, было "['A', 'B']", стало ['A', 'B']
            return [str(x).strip() for x in lst if str(x).strip()] # перебираем элементы из списка, приводим к строке, удаляем пробелы и тп, берём элемент только если после strip() строка не пуста
        except Exception:
            pass

    return [p.strip() for p in re.split(r"[;,/|]", s) if p.strip()] # запасной вариант, если не получилось разбить строку до этого

In [ ]:
df['speciality'] = df['speciality'].apply(to_list)
new_df = df.explode('speciality')
new_df

In [9]:
KIDS_ADJ_RE = re.compile(r"\b(детск\w*|педиатрическ\w*)\b", flags=re.IGNORECASE) # компилируем объект регулярного выражения для последующего использования, игнорируем регистр
PEDIATR_ONLY_RE = re.compile(r"\bпедиатр\b", flags=re.IGNORECASE)

def clean_kids_flag(spec: str):
    if not isinstance(spec, str): # если каким-то образом в функцию попало не строковое значение (например NaN), просто возвращаем как есть и is_kids=False
        return spec, False

    s = spec.strip()
    has_kids_adj = bool(KIDS_ADJ_RE.search(s)) # если нашли нашу регулярку, то флаг принимает значение True
    has_pediatr = bool(PEDIATR_ONLY_RE.search(s)) # если нашли нашу регулярку, то флаг принимает значение True

    # вырезаем только прилагательные-маркеры, просто педиатров не трогаем
    if has_kids_adj:
        s = KIDS_ADJ_RE.sub("", s) # если нашли прилагательные-маркеры (из первого флага), вырезаем их из названия специальности

    # чистим дефисы/тире и лишние пробелы, которые могли остаться, убираем висячие дефисы вокруг мест вырезки
    s = re.sub(r"\s{2,}", " ", s).strip(" ,.;-–—")

    is_kids = has_kids_adj or has_pediatr

    return s, is_kids

new_df[["speciality_clean", "is_kids"]] = (new_df["speciality"].apply(clean_kids_flag).apply(pd.Series))
new_df["is_kids"] = new_df["is_kids"].astype(bool)

In [ ]:
new_df

## Топ-20 врачей (учитываются только врачи для взрослых)

In [11]:
adults_df = new_df[new_df['is_adults'] == True]

In [ ]:
counts = (adults_df['speciality_clean'].dropna().value_counts().head(20)) # считаем, сколько раз встречается каждая специальность, берём топ-20 самых частых

counts = counts.sort_values(ascending=True) # сортируем по возрастанию
fig, ax = plt.subplots(figsize=(14, 6))
ax.barh(counts.index, counts.values, color= '#5e3c6e')

for y, v in enumerate(counts.values): # проходим по всем столбцам. y - номер строки (позиция по вертикали), v — значение (длина столбика)
    ax.text(v + max(counts.values)*0.01, y, str(v), va="center") # рисуем подпись с числом справа от столбика. v + max(counts.values)*0.01 — текст чуть правее конца столбика. y — вертикальная позиция подписи ровно напротив столбика. va="center" — выравнивание текста по вертикали по центру столбика

ax.set_xlabel('Количество')
ax.set_ylabel('Специальность')
ax.set_title('Топ-20 специальностей для взрослых людей')

plt.tight_layout()
plt.show()

## Топ-20 врачей (учитываются толькое детские)

In [14]:
kids_df = new_df[new_df['is_kids'] == True]

In [ ]:
counts = (kids_df['speciality_clean'].dropna().value_counts().head(20)) # считаем, сколько раз встречается каждая специальность, берём топ-20 самых частых

counts = counts.sort_values(ascending=True) # сортируем по возрастанию
fig, ax = plt.subplots(figsize=(14, 6))
ax.barh(counts.index, counts.values, color= '#5e3c6e')

for y, v in enumerate(counts.values): # проходим по всем столбцам. y - номер строки (позиция по вертикали), v — значение (длина столбика)
    ax.text(v + max(counts.values)*0.01, y, str(v), va="center") # рисуем подпись с числом справа от столбика. v + max(counts.values)*0.01 — текст чуть правее конца столбика. y — вертикальная позиция подписи ровно напротив столбика. va="center" — выравнивание текста по вертикали по центру столбика

ax.set_xlabel('Количество')
ax.set_ylabel('Специальность')
ax.set_title('Топ-20 детских специальностей')

plt.tight_layout()
plt.show()

## Топ-30 редких специальностей среди взрослых

In [23]:
freq = adults_df['speciality_clean'].value_counts() # считаем частоту встречаний каждого значения
freq_sorted = freq.sort_values(ascending=False) # сортируем по убыванию
rare = freq[freq <= 10].sort_values(ascending=True) # считаем редкими тех, кто встречается не более 10 раз

In [ ]:
plt.figure(figsize=(15, 8))
plt.barh(rare.index, rare.values, color= '#5e3c6e')

for y, v in enumerate(rare.values):
    plt.text(v + (rare.values.max()*0.02 if rare.values.max()>0 else 0.1), y, str(v), va="center")

plt.xlabel("Количество")
plt.ylabel("Специальность")
plt.title("Топ-30 редких специальностей")

plt.tight_layout()
plt.show()

## Топ-30 редких специальностей среди детей

In [ ]:
freq = kids_df['speciality_clean'].value_counts() # считаем частоту встречаний каждого значения
freq_sorted = freq.sort_values(ascending=False) # сортируем по убыванию
rare = freq[freq <= 10].sort_values(ascending=True) # считаем редкими тех, кто встречается не более 10 раз

In [ ]:
plt.figure(figsize=(15, 8))
plt.barh(rare.index, rare.values, color= '#5e3c6e')

for y, v in enumerate(rare.values):
    plt.text(v + (rare.values.max()*0.02 if rare.values.max()>0 else 0.1), y, str(v), va="center")

plt.xlabel("Количество")
plt.ylabel("Специальность")
plt.title("Топ-30 редких специальностей среди детей")

plt.tight_layout()
plt.show()

## Доли детских врачей

In [ ]:
ct = pd.crosstab(new_df["speciality_clean"], new_df["is_kids"]) # строим перекрёстную таблицу
colors = ["#5e3c6e", "#f59e42"]

share = ct.div(ct.sum(1), axis=0).loc[ct.sum(1).nlargest(20).index] # делим каждую строку ct на её собственную сумму, берём индексы топ-20 самых многочисленных специальностей по общему числу врачей
ax = share.plot(kind="barh", stacked=True, figsize=(10, 8), color=colors, edgecolor="white")

leg = ax.legend(title="is_kids", bbox_to_anchor=(1.02, 0.5), frameon=False)

## Исследование ко-встречаемости

In [ ]:
flat = [sp for L in df["speciality"] for sp in L] # берём каждую строку (L — это список спец. врача) и из каждого такого списка берём каждую спец. (sp). В итоге flat — это один длинный список всех спец. по всем врачам
vc = pd.Series(flat).value_counts() # считаем, сколько раз каждая спец. встречается. получаем частоты по всем спец.

TOPN = 25 # берем только 25 самых популярных специальностей
topN = vc.head(TOPN).index # получаем эти специальности

# специальный счётчик из модуля collections. он ведёт частоты объектов как словарь: ключ -> сколько раз встретился
pair_counter = Counter() # сколько раз встретилась каждая спец.

for L in df["speciality"]:
    S = [sp for sp in L if sp in topN] # оставляем только те спец. этого врача, которые входят в наш топ-25
    if not S:
        continue
    S = sorted(set(S)) # set(S), чтобы внутри одной строки не учитьывать два раза одну и ту же специальность. сортируем для того, чтобы был единый лексикографический порядок
    for a, b in combinations(S, 2): # берем все комбинации по 2
        pair_counter[(a,b)] += 1
        pair_counter[(b,a)] += 1 # делаем матрицу симметричной

C = pd.DataFrame(0, index=topN, columns=topN, dtype=int) # создаём квадратную таблицу, всю забитую нулями
for (a,b), c in pair_counter.items():
    C.loc[a,b] = c # заполняем ячейки числами«сколько раз пара (a,b) встретилась у врачей
np.fill_diagonal(C.values, 0) # чтобы не сбивать с толку на диагонали поставим значение 0

masked = np.ma.masked_where(C.to_numpy() == 0, C.to_numpy()) # всё, где матрица C равна нулю, помечаем как маску (невидимые значения)
cmap = plt.cm.viridis.copy() # берем копию колормэпа viridis, чтобы можно было менять его свойства не трогая глобальный)
cmap.set_bad('#f2f2f2') # так рисуем нули

fig, ax = plt.subplots(figsize=(11, 9))
im = ax.imshow(masked, cmap=cmap, aspect="auto", vmin=1, vmax=C.values.max()) # разрешаем оси растягиваться по размеру фигуры, нижняя граница шкалы = 1, чтобы 0 точно не окрашивался как данные, верхняя граница шкалы = максимум в C
ax.set_xticks(range(C.shape[1])); ax.set_yticks(range(C.shape[0])) # ставим подписи на каждую колонку и строку матрицы
ax.set_xticklabels(C.columns, rotation=90); ax.set_yticklabels(C.index) # подписываем их значения
ax.set_title(f"Ко-встречаемость специальностей")
plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()